In [1]:
install.packages("tensorflow")

also installing the dependencies ‘rprojroot’, ‘RcppTOML’, ‘here’, ‘png’, ‘whisker’, ‘config’, ‘reticulate’, ‘tfruns’, ‘tfautograph’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library(reticulate)
path_to_python <- install_python()
virtualenv_create("r-reticulate", python = path_to_python)

Installing pyenv ...
Done! pyenv has been installed to '/home/datalore/.local/share/r-reticulate/pyenv/bin/pyenv'.


+ '/home/datalore/.local/share/r-reticulate/pyenv/libexec/pyenv' install --skip-existing 3.9.17



ERROR: ERROR: Error: installation of Python 3.9.17 failed


In [3]:
library(tensorflow)
install_tensorflow(envname="r-reticulate")

Using Python: /opt/anaconda3/bin/python3.8
Creating virtual environment 'r-reticulate' ... 
Done!
Installing packages: 'pip', 'wheel', 'setuptools', 'numpy'
Virtual environment 'r-reticulate' successfully created.
Using virtual environment 'r-reticulate' ...


+ '/opt/anaconda3/bin/python3.8' -m venv '/home/datalore/.virtualenvs/r-reticulate'

+ '/home/datalore/.virtualenvs/r-reticulate/bin/python' -m pip install --upgrade --no-user 'pip' 'wheel' 'setuptools' 'numpy'

+ '/home/datalore/.virtualenvs/r-reticulate/bin/python' -m pip install --upgrade --no-user --ignore-installed 'tensorflow==2.11.*'



In [4]:
install.packages("keras")
library(keras)
install_keras(envname="r-reticulate")

Using virtual environment 'r-reticulate' ...

Installation complete.



also installing the dependency ‘zeallot’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

+ '/home/datalore/.virtualenvs/r-reticulate/bin/python' -m pip install --upgrade --no-user --ignore-installed 'tensorflow==2.11.*' 'tensorflow-hub' 'tensorflow-datasets' 'scipy' 'requests' 'Pillow' 'h5py' 'pandas' 'pydot'



In [5]:
library(tensorflow)
tf$constant("Witaj Tensoflow!")

tf.Tensor(b'Witaj Tensoflow!', shape=(), dtype=string)

In [9]:
library(tensorflow)
library(keras)

In [11]:
#pobranie zbioru danych
c(c(train_images,train_labels),c(test_images,test_labels)) %<-% dataset_mnist()

train_labels <- train_labels[1:1000]
test_labels <- test_labels[1:1000]

train_images <- train_images[1:1000,,] %>% array_reshape(dim=c(1000,784))/255
test_images <- test_images[1:1000,,] %>% array_reshape(dim=c(1000,784))/255

In [12]:
#definicja modelu
create_model <- function() {
    model <- keras_model_sequential() %>%
        layer_dense(512, activation = 'relu', input_shape=shape(784)) %>%
        layer_dropout(0.2) %>%
        layer_dense(10)
    model %>% compile(
        optimizer = 'adam',
        loss = loss_sparse_categorical_crossentropy(from_logits = TRUE),
        metrics = list(metric_sparse_categorical_accuracy())
    )
    model

}
model <- create_model()
summary(model)

Model: "sequential"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 dense_1 (Dense)                    (None, 512)                     401920      
 dropout (Dropout)                  (None, 512)                     0           
 dense (Dense)                      (None, 10)                      5130        
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
________________________________________________________________________________


In [14]:
checkpoint_path <- "training_1/cp.ckpt"
checkpoint_dir <- fs::path_dir(checkpoint_path)

cp_callback <- callback_model_checkpoint(
    filepath = checkpoint_path,
    save_weights_only = TRUE,
    verbose=1
)

model %>% fit(
    train_images,
    train_labels,
    epochs = 10,
    validation_data = list(test_images,test_labels),
    callbacks = list(cp_callback)
)

In [15]:
fs::dir_tree(checkpoint_dir)

training_1
├── checkpoint
├── cp.ckpt.data-00000-of-00001
└── cp.ckpt.index


In [16]:
model <- create_model()
untrained_results <- model %>% evaluate(test_images,test_labels,verbose=2)
cat("wyniki modelu niewytrenowanego:\n")

wyniki modelu niewytrenowanego:


In [17]:
untrained_results

loss 2.39550733566284 sparse_categorical_accuracy 0.11599999666214

In [18]:
load_model_weights_tf(model,checkpoint_path)

restored_model <- model %>% evaluate(test_images,test_labels,verbose=2)
cat("wyniki modelu odtworzonego:\n")

wyniki modelu odtworzonego:


In [19]:
restored_model

loss 0.428747147321701 sparse_categorical_accuracy 0.865000009536743

In [ ]:
#opcje checkpointów
checkpoint_path <- "training_2/cp-list{epoch:04d}.ckpt"
checkpoint_dir <- fs::path_dir(checkpoint_path)

batch_size <- 32

cp_callback <- callback_model_checkpoint(
    filepath = checkpoint_path,
    save_weights_only = TRUE,
    verbose=1
    save_freq = 5*batch_size
)

model<-create_model()

model %>% fit(
    train_images,
    train_labels,
    epochs = 50,
    batch_size = batch_size,
    validation_data = list(test_images,test_labels),
    callbacks = list(cp_callback),
    verbose=0
)

In [ ]:
fs::dir_tree(checkpoint_dir)

In [ ]:
latest <- tf$train$latest_checkpoint(checkpoint_dir)
latest

In [ ]:
model <- create_model()
load_model_weights_tf(model,latest)
model %>% evaluate(test_images,test_labels,verbose=0)

In [ ]:
#manualny zapis wag
save_model_weights_tf(model,'./checkpoints/my_checkpoint')

model <- create_model()
load_model_weights_tf(model,'./checkpoints/my_checkpoint')

retored_model <- model %>% evaluate(test_images,test_labels,verbose=2)

In [ ]:
cat("odtworzony model: \n")
restored_model

In [ ]:
#zapis całego modelu
model <-create_model()
model %>% fit(train_images,train_labels,epochs=5)
save_model_tf(model,"saved_model/my_model")